In [1]:
import numpy as np
import torch
import pandas as pd
import os

import torch.nn as nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader
from missforest import MissForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings

In [14]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
polish_companies_bankruptcy = fetch_ucirepo(id=365) 
  
# data (as pandas dataframes) 
X = polish_companies_bankruptcy.data.features 
Y = polish_companies_bankruptcy.data.targets 
  
# # metadata 
# print(polish_companies_bankruptcy.metadata) 
  
# # variable information 
# print(polish_companies_bankruptcy.variables) 

# data dimension
print(X.shape)
print(X.head())


(43405, 65)
   year        A1       A2       A3      A4       A5       A6        A7  \
0     1  0.200550  0.37951  0.39641  2.0472  32.3510  0.38825  0.249760   
1     1  0.209120  0.49988  0.47225  1.9447  14.7860  0.00000  0.258340   
2     1  0.248660  0.69592  0.26713  1.5548  -1.1523  0.00000  0.309060   
3     1  0.081483  0.30734  0.45879  2.4928  51.9520  0.14988  0.092704   
4     1  0.187320  0.61323  0.22960  1.4063  -7.3128  0.18732  0.187320   

        A8      A9  ...       A55       A56      A57      A58       A59  \
0  1.33050  1.1389  ...  348690.0  0.121960  0.39718  0.87804  0.001924   
1  0.99601  1.6996  ...    2304.6  0.121300  0.42002  0.85300  0.000000   
2  0.43695  1.3090  ...    6332.7  0.241140  0.81774  0.76599  0.694840   
3  1.86610  1.0571  ...   20545.0  0.054015  0.14207  0.94598  0.000000   
4  0.63070  1.1559  ...    3186.6  0.134850  0.48431  0.86515  0.124440   

      A60     A61      A62     A63      A64  
0  8.4160  5.1372   82.658  4.4158   7.4

In [15]:
X.loc[:,'year'] = X.loc[:,'year'].astype('category')
X = pd.get_dummies(X, columns=['year'], prefix='year')

In [16]:
Omega = X.isna().to_numpy()
Omega = Omega.astype(int)

X_ZI = X.copy(deep=True)
X_ZI = X_ZI.fillna(0)
X_ZI = X_ZI.to_numpy()
scaler = StandardScaler()
scaler.fit(X_ZI)
X_ZI = scaler.transform(X_ZI)
Y = Y.to_numpy().reshape(-1)

In [18]:
X_ZI_train, X_ZI_test, Y_train, Y_test, Omega_train, Omega_test = train_test_split(X_ZI, Y, Omega, test_size=0.2, random_state=42)

In [19]:
X_ZI_train = torch.tensor(X_ZI_train, dtype=torch.float32)
X_ZI_test = torch.tensor(X_ZI_test, dtype=torch.float32)

Omega_train = torch.tensor(Omega_train, dtype=torch.float32)
Omega_test = torch.tensor(Omega_test, dtype=torch.float32)

Y_train = torch.tensor(Y_train, dtype=torch.long)

In [20]:
lr = 0.001
epochs = 200
l1_lambda = 0

class TwoStreamModel(nn.Module):
    def __init__(self):
        super().__init__()
        embedding_dim = 5
        
        # The embedding layer
        self.embedding = nn.Sequential(
            nn.Linear(69, 20),  
            nn.ReLU(),
            nn.Linear(20, embedding_dim),  
            nn.ReLU()
        )
        
        # Combined network for the layers after embedding
        self.combined = nn.Sequential(
            nn.Linear(69 + embedding_dim, 140),
            nn.ReLU(),
            nn.Linear(140, 140),
            nn.ReLU(),
            nn.Linear(140, 70),
            nn.ReLU(),
            nn.Linear(70, 70),
            nn.ReLU(),
            nn.Linear(70, 70),
            nn.ReLU(),
            nn.Linear(70, 2)  # Output layer
        )
    
    def forward(self, x, omega):
        # Apply the embedding layer to omega
        embedded = self.embedding(omega)
        
        # Concatenate the embedding with x
        combined_features = torch.cat((x, embedded), dim=1)
        
        # Apply the combined network
        final_out = self.combined(combined_features)
        
        return final_out
    

model_PE = TwoStreamModel()
PA_train_data = TensorDataset(X_ZI_train, Omega_train, Y_train)
PA_train_loader = DataLoader(dataset = PA_train_data, batch_size=64, shuffle=True)

optimizer = optim.Adam(model_PE.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
class_weights = torch.tensor([0.05, 0.95], dtype=torch.float32)
loss_fn = nn.CrossEntropyLoss()    # (weight=class_weights)

for epoch in range(epochs):

    for x_batch, omega_batch, y_batch in PA_train_loader:
        optimizer.zero_grad()
        pred = model_PE(x_batch, omega_batch)
        loss = loss_fn(pred, y_batch)

        # L1 penalty
        l1_penalty = 0
        for param in model_PE.parameters():
            l1_penalty += torch.sum(torch.abs(param))
        # Add L1 penalty to the loss
        loss += l1_lambda * l1_penalty

        loss.backward()
        optimizer.step()

    scheduler.step()

    if epoch % 10 == 9:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

_, labels = torch.max(model_PE(X_ZI_test, Omega_test), dim=1)
pred = labels.detach().numpy()
np.mean(np.abs(pred - Y_test))

Epoch 9, Loss: 0.08916226029396057
Epoch 19, Loss: 0.07397502660751343
Epoch 29, Loss: 0.10288068652153015
Epoch 39, Loss: 0.11827834695577621
Epoch 49, Loss: 0.021093865856528282
Epoch 59, Loss: 0.0789480060338974
Epoch 69, Loss: 0.08012749254703522
Epoch 79, Loss: 0.06209595501422882
Epoch 89, Loss: 0.11358650773763657
Epoch 99, Loss: 0.04667492210865021
Epoch 109, Loss: 0.012480970472097397
Epoch 119, Loss: 0.007120020687580109
Epoch 129, Loss: 0.06877294182777405
Epoch 139, Loss: 0.04933222010731697
Epoch 149, Loss: 0.04728896543383598
Epoch 159, Loss: 0.02614790014922619
Epoch 169, Loss: 0.1637321412563324
Epoch 179, Loss: 0.10963525623083115
Epoch 189, Loss: 0.06982632726430893
Epoch 199, Loss: 0.019491545855998993


np.float64(0.03985715931344315)

In [21]:
_, labels2 = torch.max(model_PE(X_ZI_train, Omega_train), dim=1)
pred2 = labels2.detach().numpy()
print(np.mean(np.abs(pred2 - Y_train.detach().numpy())))
print(sum(pred2 == 1))

0.017624697615482087
1094
